In [11]:
def download_stations(url='https://apex.oracle.com/pls/apex/raspberrypi/weatherstation/getallstations', retry = 10):
    attempts = 0
    while attempts <= retry:
        attempts += 1
        try:
            response = urllib.request.urlopen(url)
            stations = eval(response.read().decode())
            break
        except:
            response = urllib.request.urlopen(url)
            stations = eval(response.read().decode())
    return stations

In [12]:
def store_stations(stations, db):
    with sqlite3.connect(db) as conn:
        conn.executescript("""
        PRAGMA foreign_keys = ON;
        
        CREATE TABLE IF NOT EXISTS Stations(
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        latitude REAL NOT NULL,
        longitude REAL NOT NULL
        );

        CREATE TABLE IF NOT EXISTS Measurements(
        air_pressure REAL,
        air_quality REAL,
        ambient_temp REAL,
        created_by TEXT,
        created_on TEXT,
        ground_temp REAL,
        humidity REAL,
        id INTEGER PRIMARY KEY,
        rainfall INTEGER,
        reading_timestamp TEXT,
        updated_by TEXT,
        updated_on TEXT,
        stations_id INT,
        wind_direction INTEGER,
        wind_gust_speed INTEGER,
        wind_speed INTEGER,
        FOREIGN KEY (stations_id) REFERENCES Stations(id)
        );
        """)
        for station in stations:
            conn.execute("""
            INSERT OR IGNORE INTO Stations
            VALUES(:weather_stn_id, :weather_stn_name, :weather_stn_lat, :weather_stn_long)""", station)
        conn.commit()
    return conn    

In [13]:
def download_measurements(station, db, retry = 10):
    url = 'https://apex.oracle.com/pls/apex/raspberrypi/weatherstation/getlatestmeasurements/' + str(station)
    attempts = 0
    while attempts <= retry:
        attempts += 1
        try:
            response = urllib.request.urlopen(url)
            measurements = eval(response.read().decode())
            break
        except IncompleteRead:
            response = urllib.request.urlopen(url)
            measurements = eval(response.read().decode())
    measurements = measurements['items']
    if measurements:
        with sqlite3.connect(db) as conn:
            for measurement in measurements:
                for key in measurement.keys():
                    if key == 'weather_stn_id':
                        conn.execute("INSERT OR IGNORE INTO Measurements(stations_id) VALUES(:weather_stn_id)", measurement)
                    else:
                        query = f"INSERT OR IGNORE INTO Measurements({key}) VALUES (?)"
                        conn.execute(query, (measurement[key],))
        conn.commit()
    return measurements

In [14]:
def download_all_measurements():
    from sys import argv
    db = argv[1]
    import urllib
    import sqlite3
    stations = download_stations()['items']
    store_stations(stations, db)
    for station in stations:
        stations_id = station['weather_stn_id']
        download_measurements(stations_id, db)

        
    if __name__ == '__main__':
download_all_measurements()

NameError: name 'IncompleteRead' is not defined